# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Общая информация

Импортируем библиотеки

In [1]:
import pandas as pd
from pymystem3 import Mystem
import numpy as np

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

1. Таблица содержит более 20 тысяч строк в 12 колонках. Названия колонок записаны в нижнем и "змеином" регистре и все латиницей, что корректно. Переименование не потребуется. 
2. Есть пропуски, их нужно обработать. 
3. Нужно проверить гипотезы, связанные с семейным положением, наличием детей, доходом и фактом погашения кредита, а значит в первую очередь я смотрю на колонки children, family_status, family_status_id, total_income, debt. У колонок дети, идентификатор семейного положения и долг целочисленный тип данных. Предположу, что для индентификатора семейного положения расшифровка будет в колонке семейное положение (строковый тип данных), а для долга значения 0 и 1 как факт погашения и непогашения долга. У колонки доход вещественный тип данных, а значит с ними можно производить математические операции.

## Шаг 2. Предобработка данных

В предобработке данных я планирую провести:
1. Обработка пропусков. В зависимости от их количества буду заполнять пропуски или удалять.
2. Замена типа данных. Заменю тип данных в колонке доход
3. Обработка дубликатов. Найду дубликаты и обработаю
4. Лемматизация. Так как одна из задач связана с колонкой цель кредита, проведу лемматизацию в колонке 
5. Категоризация. Категоризирую заемщиков по группам для дальнейшего исследования.

### Обработка пропусков

In [3]:
data.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Методом isna().sum() подсчитываю пропуски в таблице. Их 2174 в колонках трудовой стаж и доход. Меня интересует колонка доход. Пропуски составляют около 10% от всех данных. Это существенный процент, если их игнорировать, расчеты могут быть некорретктны. Точно определить причину пропусков не могу, при наличии возможности уточнила бы у разработчиков, а пока предположу, что пропуски могут быть связаны с тем, что: 1) заемщик не имеет дохода вообще, либо подтвержденного дохода; 2) возможно, данные о доходах попадают в колонку из сведений по зарплатной/пенсионной карте этого банка. Если у заемщика другой банк, о его доходах нет сведений. Ни одно из предположений не исключает, что заемщик тем не менее в состоянии выплачивать кредит. А значит, я бы не стала удалять пропуски, а заменила значение на среднее по типу занятости. 

Пишу функцию для замены значения по типу занятости (income_type) и методом apply применяю ее к столбцу доход (total_income). 

In [5]:
data_grouped = data.groupby('income_type')['total_income'].mean()
 
def func(row):
    if pd.isna(row['total_income']):
        return data_grouped.loc[row['income_type']]
    return row['total_income']
 
data['total_income'] = data.apply(func, axis=1)

Для проверки снова вывожу .isna().sum(). 

In [7]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

В столбце total_income пропусков нет.

**Вывод**

При обработке пропусков были найдены и обработаны 2174 пропуска в колонке доход. Пропуски были заполнены средними значениями по типу дохода. В этой колонке тип данных float. Копейки из этой колонки не существенно влияют на расчеты, но в анализе из-за них могут быть ошибки в коде. Поэтому я заменю тип данных на int

### Замена типа данных

In [177]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Так как известно, в какой тип значений нужно перевести,для изменения типа данных использую метод .astype(). Проверяю результат, вызвав info()

### Обработка дубликатов

In [8]:
data.duplicated().sum()

54

Методом .duplicated().sum() нахожу количество дубликатов в таблице. Их 54. Это менее 1% данных. Удалю явные дубликаты методом drop_duplicates()

In [9]:
data = data.drop_duplicates()
data.duplicated().sum()


0

Проверяю наличие дубликатов, снова вызвав .duplicated().sum(). Дубликатов нет.

Отдельно смотрю статистику значений по нужным мне столбцам.

In [180]:
data['family_status'].value_counts()
data['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

 Значение в идентификаторе семейного положения действительно соответствует расшифровке в колонке семейное положение, а значит для расчетов я могу использовать идентификатор. Так удобнее. Самое распространенное значение "женат/замужем".

In [10]:
data['children'].value_counts()


 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Самое распространенное значение в колонке дети - 0 (детей нет), однако есть странные значения, вроде -1, их количество(47) меньше 1% от общих данных. Рискну предположить, что это опечатка (вместо 1), как 20 (вместо 2). Я заменю их методом replace().

In [11]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)
data['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Метод добавил значения из -1 и 20 в соответствующие 1 и 2

In [12]:
data['total_income'].value_counts()


161380.260488    1077
202417.461462     503
137127.465690     394
170898.309923     145
499163.144947       2
                 ... 
175057.266090       1
101516.604975       1
239154.168013       1
165009.733021       1
189255.286637       1
Name: total_income, Length: 19355, dtype: int64

**Вывод**

При обработке дубликатов выяснилось, что явных дубликатов в датасете 54, что составляет менее 1% данных. Явные дубликаты были удалены методом drop_dulicates(). При подробном изучении значений в колонках были обнаружены нехарактерные значения, как то -1 в колонке дети. Они были заменены на предполагаемые с учетом того, что процент таких "странных" данных невелик и не сможет серьезно повлиять на исследование.

### Лемматизация

Далее выделяю леммы в колонке цели методом .lemmatize() 

In [13]:
m = Mystem()
data['id_purpose'] = data['purpose'].apply(m.lemmatize)
data['id_purpose'].head()

0                 [покупка,  , жилье, \n]
1       [приобретение,  , автомобиль, \n]
2                 [покупка,  , жилье, \n]
3    [дополнительный,  , образование, \n]
4               [сыграть,  , свадьба, \n]
Name: id_purpose, dtype: object

**Вывод**

Надо помнить, что этот метод берет значения строки. Чтобы он брал значения столбца, я помещаю его в apply() и применяю его к столбцу. Результаты лемматизации сохраняю в отдельный столбец для дальнейших подсчетов. Для проверки вывожу методом .head(). Леммы выделились.

### Категоризация данных

Для исследования я буду разбивать заемщиков по следующим категориям:
- Наличие детей (с детьми, без детей)
- Семейное положение (женаты, гражданский брак, неженатые, в разводе, вдовец/вдова)
- Цели кредита (жилье, автомобиль, другое)
- Доход (до 107т, от 107 до 146т, более 146т )
(Для деления на категории по доходу я воспользовалась методом .describe() по подсказке преподавателя по проектам. У нас этого метода еще не было. Чтобы группы были относительно равномерными по количеству, границами для меня служили метки 25%,50% и 75%)

In [14]:
def children_group(children):
        if children == 0:
            return 'без детей'
        return 'с детьми'
 
children_group(2)

'с детьми'

Пишу функцию для категоризации по наличию детей. Проверяю работу функции. Работает

In [15]:
data['children_cat'] = data['children'].apply(children_group)
data['children_cat'].value_counts() 

без детей    14107
с детьми      7364
Name: children_cat, dtype: int64

Сохраняю результаты в новом столбце, передав функцию в .apply(). Методом .value_counts() вывожу уникальные значения столбца. На данном этапе заключаю, что заемщиков без детей почти в два раза больше, чем заемщиков с детьми.

In [16]:
def status_group(family_status_id):
    if family_status_id == 0:
        return 'женатые'
    if family_status_id == 1:
        return 'гражданский брак'
    if family_status_id == 2:
        return 'вдовец/вдова'
    if family_status_id == 3:
        return 'в разводе'
    return 'неженатые'
    

status_group(4)
data['status_cat'] = data['family_status_id'].apply(status_group)
data['status_cat'].value_counts()  

женатые             12344
гражданский брак     4163
неженатые            2810
в разводе            1195
вдовец/вдова          959
Name: status_cat, dtype: int64

Пишу функцию для категоризации заемщиков по семейному положению, проверяю работу функции и передаю ее столбцу идентификатор семейного положения. Результаты категоризации сохраняю в отдельном столбце status_cat. Проверяю уникальные значения. Больше всего заемщиков в категории "женатые", их почти в три раза больше, чем в следующей по значению категории "гражданский брак". Меньше всего заемщиков в категории "вдовец/вдова"

In [17]:
def purpose_group(id_purpose):
    if 'жилье' in id_purpose or 'недвижимость' in id_purpose:
        return 'недвижимость'
    if 'автомобиль' in id_purpose:
        return 'автомобиль'
    return 'другое'
  
purpose_group('жилье')
data['purpose_cat'] = data['id_purpose'].apply(purpose_group)
data['purpose_cat'].value_counts()  

недвижимость    10814
другое           6349
автомобиль       4308
Name: purpose_cat, dtype: int64

Для категоризации по целям кредита были использованы результаты лемматизации. Они были сохранены в столбце id_purpose. Применена функция и выведены уникальные значения. Больше всего заемщиков в категории цель кредита - недвижимость.

Для того, чтобы понять, как варьируется доход, какой средний заработок, применю метод desribe() к колонке доход

In [19]:
data['total_income'].describe()

count    2.147100e+04
mean     1.674221e+05
std      9.802416e+04
min      2.066726e+04
25%      1.076553e+05
50%      1.518987e+05
75%      2.024175e+05
max      2.265604e+06
Name: total_income, dtype: float64

Обозначу примерные границы доходов - 107 000 и 146 000. Напишу функцию для разбиения заемщиков на категории

In [20]:
def income_group(income):
    if income <= 107000:
        return 'доход до 107'
    if 107000 < income <= 146000:
        return 'доход до 146'
    return 'доход больше 146'
 
data['income_cat'] = data['total_income'].apply(income_group)
data['income_cat'].value_counts()  


доход больше 146    11279
доход до 107         5307
доход до 146         4885
Name: income_cat, dtype: int64

Результаты сохраняю в отдельный столбец. Вывожу статистику по значениям. Больще всего заемщиков с доходом более 146 000

**Вывод**

В предобработке данных провела все операции согласно плану
1. Обработка пропусков. Пропуски в колонке доход были заполнены средними значениями по типу дохода
2. Замена типа данных. Тип данных в колонке доход заменен на целочмсленный
3. Обработка дубликатов. Дубликаты найдены. Их 54. Удалены
4. Лемматизация. Проведена, результаты сохранены в отдельном столбце
5. Категоризация. Категоризирую заемщиков по группам для дальнейшего исследования.

Предобработка данных позволила привести датасет в состояние, которое упрощает дальнейшее решение задачи.

## Шаг 3. Наличие зависимости между признаками и фактом возвращения кредита в срок

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сводной таблицей выведем значения в категориях и их доли (через среднее)

In [21]:
children_pivot = data.pivot_table(index=['children_cat'], values=['debt'],aggfunc=[np.mean, len])
children_pivot

,mean,len
,debt,debt
children_cat,,
без детей,0.075353,14107
с детьми,0.092070,7364


Итак, доля должников без детей - 7,5%, должников с детьми - 9,2%. 

 **Вывод**

Доля должников не превышает 10% от общего количества заемщиков в этой категории. Более того доля должников с детьми на почти 2% выше аналогичных без детей (без детей - 7,5%, с детьми - 9,2%. ) Таким образом заключаю, что наличие детей не влияет положительно на платежеспособность.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
status_pivot = data.pivot_table(index=['status_cat'], values=['debt'],aggfunc=[np.mean, len])
status_pivot

,mean,len
,debt,debt
status_cat,,
в разводе,0.071130,1195
вдовец/вдова,0.065693,959
гражданский брак,0.093202,4163
женатые,0.075421,12344
неженатые,0.097509,2810


**Вывод**

Вывожу сводную таблицу в категории семейное положение. В столбце среднее - доля должников соответсвенно положению. 
Наиболее высокие значения в категориях "неженатые" и "гражданский брак", 9,7% и 9,3% соответственно. Почти на 2% ниже доля должников в категориях "женатые" и "в разводе", 7,5% и 7,1%. И самый маленький процент должников в категории "вдовец/вдова". Заключаю, что наиболее платежеспособная категория заемщиков "вдовец/вдова". Доля должников в этой категории на 3,2% ниже, чем в наименее платежеспособной "неженатые".

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
income_pivot = data.pivot_table(index=['income_cat'], values=['debt'],aggfunc=[np.mean, len])
income_pivot

,mean,len
,debt,debt
income_cat,,
доход больше 146,0.079085,11279
доход до 107,0.079706,5307
доход до 146,0.087206,4885


**Вывод**

Доли в процентах показали, что должников больше в группе, где доход больше (8,7%). В группах в меньшим доходом доли должников 7,9%. Между ними разница менее одного процента (0,8%)

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
purpose_pivot = data.pivot_table(index=['purpose_cat'], values=['debt'],aggfunc=[np.mean, len])
purpose_pivot

,mean,len
,debt,debt
purpose_cat,,
автомобиль,0.093547,4308
другое,0.087573,6349
недвижимость,0.072314,10814


**Вывод**

Максимальное значение доли должников в группе цель кредита автомобиль - 9,3%. Минимальное в категории "недвижимость" - 7,2%. Доля должников, берущих кредит под покупку жилья более, чем на 2 % ниже, чем доля должников среди заемщиков, которые берут кредит на покупку автомобиля.

## Шаг 4. Общий вывод

В исследовании для кредитного отдела банка была поставлена задача найти взаимосвязи платежеспособности и фактом наличия детей, семейным положением, а также уровнем дохода и цели кредита. Для исследования предобработка данных была проведена таким образом, что поиски взаимосвязей упростились. Предобработка включала в себя обработку пропусков и дубликатов, лемматизацию и категоризацию. Это добавило дополнительных столбцов в датасете и благодаря им стало возможно построение сводных таблиц для решения задачи и ответов на вопросы. В каждой категории заемщиков были найдены доли должников и их можно сравнивать, чтобы сделать выводы. Доля должников во всех группах не превышает 10% от общего количества заемщиков
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
Доля должников с детьми на почти 2% выше аналогичных без детей (без детей - 7,5%, с детьми - 9,2%. ) Таким образом заключаю, что наличие детей не влияет положительно на платежеспособность.
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
Наиболее высокие значения в категориях "неженатые" и "гражданский брак", 9,7% и 9,3% соответственно. Почти на 2% ниже доля должников в категориях "женатые" и "в разводе", 7,5% и 7,1%. И самый маленький процент должников в категории "вдовец/вдова". Заключаю, что наиболее платежеспособная категория заемщиков "вдовец/вдова". Доля должников в этой категории на 3,2% ниже, чем в наименее платежеспособной "неженатые".
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Доли в процентах показали, что должников больше в группе, где доход больше (8,7%). В группах в меньшим доходом доли должников 7,9%. Между ними разница менее одного процента (0,8%)
- Как разные цели кредита влияют на его возврат в срок?
Максимальное значение доли должников в группе цель кредита автомобиль - 9,3%. Минимальное в категории "недвижимость" - 7,2%. Доля должников, берущих кредит под покупку жилья более, чем на 2 % ниже, чем доля должников среди заемщиков, которые берут кредит на покупку автомобиля.